In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import random

random.seed()

In [ ]:
def showBars(tags, rets, totalnums):
    fig = go.Figure()
    
    for i in range(len(tags)):
        xarr = []
        yarr = []
        
        for k, v in rets[i].items():
            xarr.append(k)
            yarr.append(v / totalnums)
            
        fig.add_trace(go.Bar(
            x=xarr,
            y=yarr,
            name=tags[i],
        ))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
# 料事如神，35%的概率，对敌方2人造成106%的谋略伤害
def zfLSRS(turnNums):
    atk = 0
    for i in range(turnNums):
        if random.random() < 0.35:
            atk += 1.06
            atk += 1.06
    
    return atk

# 落雷，50%的概率，对单体（有5%的概率对己方）造成170%的谋略伤害，谋略伤害提升18%，2回合
def zfLL(turnNums):
    atk = 0
    for i in range(turnNums):
        if random.random() < 0.5:
            if random.random() >= 0.05:
                atk += 1.7
    
    return atk

def countArr(enemyindex, arr):
    atk = 0
    for i in range(len(arr)):
        if arr[i]['ei'] == enemyindex and arr[i]['lastturn'] > 0:
            atk += arr[i]['val']
    
    return atk
        
def clearArr(arr):
    if len(arr) <= 0:
        return 
    
    for i in range(len(arr)):
        if arr[i]['lastturn'] <= 0:
            del arr[i]
            clearArr(arr)
            
def procArrLastTurn(arr):
    for i in range(len(arr)):
        arr[i]['lastturn'] -=1
        
#     clearArr(arr)            
            

# 落雷，50%的概率，对单体（有5%的概率对己方）造成170%的谋略伤害，谋略伤害提升18%，2回合
def zfLL2(turnNums):
    atk = 0
    arr = []
    for i in range(turnNums):
        procArrLastTurn(arr)
        
        if random.random() < 0.5:
            if random.random() >= 0.05:
                ei = int(random.random() * 3)
                
                atkoff = countArr(ei, arr)
                atk += 1.7 * (1 + atkoff)
                
                arr.append({'ei': ei, 'lastturn': 2, 'val': 0.18})
    
    return atk

# 用武神通，2/4/6/8对2人造成75%/105%/135%/165%的谋略伤害
def zfYWST(turnNums):
    atk = 0
    for i in range(turnNums):
        if i == 1:
            atk += 0.75
            atk += 0.75
        elif i == 3:
            atk += 1.05
            atk += 1.05
        elif i == 5:
            atk += 1.35
            atk += 1.35
        elif i == 7:
            atk += 1.65
            atk += 1.65            
    
    return atk

# 沉沙诀水，40%概率，准备1回合，对2人造成水攻状态（每回合126%伤害），持续2回合
def zfCSJS(turnNums):
    atk = 0
    ready = False
    enemy = [0, 0, 0]
    for i in range(turnNums):
        for t in range(3):
            if enemy[t] > 0:
                atk += 1.26
                enemy[t] -= 1
                    
        if ready:
            noe = int(random.random() * 3)
#             cenemy = [2, 2, 2]
#             cenemy[noe] = 0
            for t in range(3):
                if enemy[t] == 0 and t != noe:
                    enemy[t] = 2
            
            ready = False
        else:
            if random.random() < 0.4:
                ready = True
    
    return atk

# 沉沙诀水 先攻，40%概率，准备1回合，对2人造成水攻状态（每回合126%伤害），持续2回合
def zfCSJS_X(turnNums):
    atk = 0
    ready = False
    enemy = [0, 0, 0]
    for i in range(turnNums):
        if ready:
            noe = int(random.random() * 3)
#             cenemy = [2, 2, 2]
#             cenemy[noe] = 0
            for t in range(3):
                if enemy[t] == 0 and t != noe:
                    enemy[t] = 2
            
            ready = False
        else:
            if random.random() < 0.4:
                ready = True
                
        for t in range(3):
            if enemy[t] > 0:
                atk += 1.26
                enemy[t] -= 1                
    
    return atk

def addStdVal(stdval, val):
    ival = int(val * 100)
    for i in range(len(stdval['val'])):
        if int(stdval['val'][i] * 100) == ival:
            stdval['weight'][i] += 1
            return stdval
    
    stdval['val'].append(val)
    stdval['weight'].append(1)
    
    return stdval

def randZF(totalNums, funcZF):
    ret = {}
    retmin = {}
    retmax = {}
    retstd = {}
    
    for t in range(8):
        ret[t + 1] = 0
        retmin[t + 1] = 99999999
        retmax[t + 1] = 0
        retstd[t + 1] = {'val': [], 'weight': []}
        
        for i in range(totalNums):
            cv = funcZF(t + 1)
            ret[t + 1] += cv
            if cv > retmax[t + 1]:
                retmax[t + 1] = cv
            if cv < retmin[t + 1]:
                retmin[t + 1] = cv
                
            addStdVal(retstd[t + 1], cv)
            
    return ret, retmin, retmax, retstd

In [ ]:
totalNums = 1000000

retLSRS, retLSRSmin, retLSRSmax, retLSRSstd = randZF(totalNums, zfLSRS)
retLL, retLLmin, retLLmax, retLLstd = randZF(totalNums, zfLL)
retLL2, retLL2min, retLL2max, retLL2std = randZF(totalNums, zfLL2)

In [ ]:
retYWST, retYWSTmin, retYWSTmax, retYWSTstd = randZF(totalNums, zfYWST)

In [ ]:
retCSJS, retCSJSmin, retCSJSmax, retCSJSstd = randZF(totalNums, zfCSJS)

In [ ]:
retCSJS_X, retCSJS_Xmin, retCSJS_Xmax, retCSJS_Xstd = randZF(totalNums, zfCSJS_X)

In [ ]:
showBars([
    '料事如神期望', 
    '落雷期望',
    '落雷考虑增攻期望',    
    '用武神通期望', 
    '沉沙诀水期望', 
    '沉沙诀水（先攻）期望'], [
    retLSRS, 
    retLL, 
    retLL2,
    retYWST, 
    retCSJS, 
    retCSJS_X], totalNums)

In [ ]:
retCSJS_Xmax

In [ ]:
showBars([
    '料事如神最小期望', 
    '料事如神最大期望',     
    '落雷最小期望',    
    '落雷最大期望',    
    '落雷考虑增攻最小期望',    
    '落雷考虑增攻最大期望',        
    '用武神通最小期望',     
    '用武神通最大期望',         
    '沉沙诀水最小期望',     
    '沉沙诀水最大期望',         
    '沉沙诀水（先攻）最小期望',
    '沉沙诀水（先攻）最大期望'    
], [
    retLSRSmin, 
    retLSRSmax,
    retLLmin, 
    retLLmax,
    retLL2min,
    retLL2max,
    retYWSTmin, 
    retYWSTmax,
    retCSJSmin, 
    retCSJSmax,
    retCSJS_Xmin,
    retCSJS_Xmax], 1)

In [ ]:
def weighted_std(values, weights):
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return np.sqrt(variance)

retLSRSstd1 = {}
retLLstd1 = {}
retLL2std1 = {}
retYWSTstd1 = {}
retCSJSstd1 = {}
retCSJS_Xstd1 = {}
for k, v in retLSRSstd.items():
    retLSRSstd1[k] = weighted_std(retLSRSstd[k]['val'], retLSRSstd[k]['weight'])
    retLLstd1[k] = weighted_std(retLLstd[k]['val'], retLLstd[k]['weight'])    
    retLL2std1[k] = weighted_std(retLL2std[k]['val'], retLL2std[k]['weight'])        
    retYWSTstd1[k] = weighted_std(retYWSTstd[k]['val'], retYWSTstd[k]['weight'])
    retCSJSstd1[k] = weighted_std(retCSJSstd[k]['val'], retCSJSstd[k]['weight'])    
    retCSJS_Xstd1[k] = weighted_std(retCSJS_Xstd[k]['val'], retCSJS_Xstd[k]['weight'])        

In [ ]:
showBars([
    '料事如神期望标准差',      
    '落雷期望标准差',        
    '落雷考虑增攻期望标准差',    
    '用武神通期望标准差',
    '沉沙诀水期望标准差',         
    '沉沙诀水（先攻）期望标准差',
], [
    retLSRSstd1, 
    retLLstd1, 
    retLL2std1,
    retYWSTstd1, 
    retCSJSstd1, 
    retCSJS_Xstd1], 1)

In [ ]:
def genOffsetArr(minval, maxval, off):
    arr = []
    for i in range(int((maxval - minval) / off)):
        arr.append(minval + i * off)
        
    arr.append(maxval)
    
    return arr



In [ ]:
arr18 = genOffsetArr(0, 18, 2)
arr18

In [ ]:
def insWeights(val, weight, offs, weights):
    for i in range(len(offs)):
        if val < offs[i]:
            weights[i] += weight
            return
    
    weights[len(offs) - 1] += weight

def rebuildWeights(vals, weights, offs):
    newweights = [0 for _ in range(len(offs))]
    
    for i in range(len(vals)):
        insWeights(vals[i], weights[i], offs, newweights)
        
    return newweights

def rebuildWeights2(vals, weights, d, offs):
    newweights = [0 for _ in range(len(offs))]
    
    for i in range(len(vals)):
        insWeights(vals[i] / d, weights[i], offs, newweights)
        
    return newweights
        


In [ ]:
retLSRSweight8 = rebuildWeights(retLSRSstd[8]['val'], retLSRSstd[8]['weight'], arr18)
retLLweight8 = rebuildWeights(retLLstd[8]['val'], retLLstd[8]['weight'], arr18)    
retLL2weight8 = rebuildWeights(retLL2std[8]['val'], retLL2std[8]['weight'], arr18)        
retYWSTweight8 = rebuildWeights(retYWSTstd[8]['val'], retYWSTstd[8]['weight'], arr18)
retCSJSweight8 = rebuildWeights(retCSJSstd[8]['val'], retCSJSstd[8]['weight'], arr18)    
retCSJS_Xweight8 = rebuildWeights(retCSJS_Xstd[8]['val'], retCSJS_Xstd[8]['weight'], arr18)        

In [ ]:
def showBars8(tags, xarr, rets, totalnums):
    fig = go.Figure()
    
    for i in range(len(tags)):
        yarr = []
        
        for j in range(len(rets[i])):
            yarr.append(rets[i][j] / totalnums)
            
        fig.add_trace(go.Bar(
            x=xarr,
            y=yarr,
            name=tags[i],
        ))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
showBars8([
    '料事如神8回合期望分布',
    '落雷8回合期望分布',        
    '落雷考虑增攻8回合期望分布',
    '用武神通8回合期望分布',
    '沉沙诀水8回合期望分布',         
    '沉沙诀水（先攻）8回合期望分布',
], arr18, [
    retLSRSweight8, 
    retLLweight8, 
    retLL2weight8,
    retYWSTweight8, 
    retCSJSweight8, 
    retCSJS_Xweight8], totalNums)

In [ ]:
def showBarsEx(tags, xarr, objs, totalnums):
    fig = go.Figure()
    
    for i in range(len(tags)):
        for k, v in objs[i].items():
            yarr1 = rebuildWeights2(v['val'], v['weight'], k, xarr)
            
            yarr = []        
            for j in range(len(yarr1)):
                yarr.append(yarr1[j] / totalnums)
            
#             fig.add_trace(go.Bar(
#                 x=xarr,
#                 y=yarr,
#                 name='{}打满{}回合'.format(tags[i], k),
#             ))            
            fig.add_trace(go.Scatter(
                x=xarr,
                y=yarr,
                mode='lines',
                name='{}打满{}回合'.format(tags[i], k),
            ))                        

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
arr3 = genOffsetArr(0, 3, 0.3)
arr3

In [ ]:
showBarsEx([
    '料事如神',
    '落雷',        
    '落雷考虑增攻',
    '用武神通',
    '沉沙诀水',         
    '沉沙诀水（先攻）',
], arr3, [
    retLSRSstd, 
    retLLstd, 
    retLL2std,
    retYWSTstd, 
    retCSJSstd, 
    retCSJS_Xstd,
], totalNums)

In [ ]:
def showBars3(tags, rets, totalnums):
    fig = go.Figure()
    
    for i in range(len(tags)):
        xarr = []
        yarr = []
        
        for k, v in rets[i].items():
            xarr.append(k)
            yarr.append(v / k / totalnums)
            
        fig.add_trace(go.Bar(
            x=xarr,
            y=yarr,
            name=tags[i],
        ))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
showBars3([
    '料事如神平均每回合期望', 
    '落雷平均每回合期望',
    '落雷考虑增攻平均每回合期望',    
    '用武神通平均每回合期望', 
    '沉沙诀水平均每回合期望', 
    '沉沙诀水（先攻）平均每回合期望'], [
    retLSRS, 
    retLL, 
    retLL2,
    retYWST, 
    retCSJS, 
    retCSJS_X], totalNums)